Encontrei o tutorial em https://medium.com/ensina-ai/prevendo-roubos-de-celulares-em-s%C3%A3o-paulo-4ee5b0845fc2

Achei muito interessante aprendi várias tecnicas como:



Caso esteja vendo no github utilize o __NBViewer__ para ver a interação dos gráficos   
Link : https://nbviewer.jupyter.org/github/eduardo-almeida/Data_Science/blob/master/seguranca_publica/Main.ipynb

### Índice:
1. Introdução
2. Importando Bibliotecas e Datasets
3. Conhecendo os Dados
4. Pré-processamento
5. Análise Exploratória
6. Prevendo Roubos de Celular 
<br>    6.1. - Modelagem ARIMA
<br>    6.2. - Validação
<br>    6.3. - Previsão


### 1. Introdução:

#### Apresentando o dataset:
__ANO_BO__ - Ano de registro da ocorrência.
<br>__DATAOCORRENCIA__: Data do roubo do celular.
<br>__PERIDOOCORRENCIA__: Período do roubo do celular.
<br>__BAIRRO__ - Bairro em que o celular foi roubado.
<br>__CIDADE__ - Cidade em que o celular foi roubado.
<br>__LATITUDE__ - Latitude em que o celular foi roubado.
<br>__LONGITUDE__ - Longitude em que o celular foi roubado.
<br>__LOGRADOURO__ - Logradouro em que o celular foi roubado.
<br>__DESCRICAOLOCAL__ - Descrição do local em que o celular foi roubado.
<br>__DESCR_TIPO_VEICULO__ - Descrição do veículo em que o celular foi roubado.


#### Objetivos:
 - Determinar como as ocorrências de roubo de celular evoluiram em relação ao tempo.
 - Determinar os períodos onde ocorrem os maiores números de roubo de celular.
 - Determinar os logradouros, bairros e regiões com maiores números de roubo de celular.
 - Determinar os locais com maiores números de roubo de celular.
 - Determinar veículos com maiores números de roubo de celular em veículo.
 - Utilizar o modelo ARIMA para prever o número de ocorrências nos próximos anos.
 
 
#### Sobre o dataset:

Desde 1995, a Secretaria da Segurança Pública divulga de forma pioneira no Brasil os indicadores criminais <br>
trimestralmente no Diário oficial.

Em decorrência da evolução da tecnologia e o amplo uso da internet, a partir de abril de 2011 esses dados <br>
passaram a ser divulgados em seu site mensalmente por região, município e unidade policial.<br>

Além desses dados, são apresentados estudos como o Perfil de Homicídio e o Perfil de Roubos além da <br>
produtividade policial.

Ampliando essa divulgação, desde 09 de maio de 2016, a SSP/SP mantém o maior portal de informações sobre <br>
segurança pública do país, disponibilizando boletins de ocorrência, inclusive os complementares, disponíveis <br>
no sistema Registro Digital de Ocorrências da Polícia Civil (RDO) desde 2003, em relação aos homicídios dolosos, <br>
latrocínios e lesão corporal seguida de morte e registrados a partir de 2013, tanto em relação a morte decorrente <br>
de oposição intervenção policial, quanto em relação aos casos de mortes suspeitas.

Na mesma data também foi disponibilizada tabela de dados referentes aos óbitos que deram entrada no Instituto <br>
Médico Legal (IML) desde 2013, ano em que foi implantado o serviço digital de Gestão de Laudos (GDL) da <br>
Superintendência da Polícia Técnico-Cientifica (SPTC), com os respectivos números dos laudos e boletins de <br>
ocorrência, data de entrada no IML, características das vítimas e número da Declaração de Óbito.

Com a evolução do serviço, em abril de 2017, foram incluídos para pesquisa todos os boletins de roubo e de <br>
furto de veículos disponíveis no RDO desde 2003, além da melhoria do serviço pela possibilidade de extração <br>
dos dados em formato excel.

Em 25 de maio de 2017, para atendimento a uma necessidade demonstrada através de inúmeros pedidos dirigidos à <br>
Secretaria, foram disponibilizados todos os boletins de ocorrência de roubo e furto onde há ao menos um celular <br>
subtraído.

Fonte: http://www.ssp.sp.gov.br/transparenciassp/Default.aspx

Os dados estão disponibilizados em http://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx

In [2]:
# IMPORTANDO BIBLIOTECAS

import pandas as pd
import glob
import re
import itertools
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import squarify

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')

import plotly.offline as py 
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot
import plotly.tools as tls # It's useful to we get some tools of plotly

from folium import Map
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'squarify'

In [3]:
# FUNÇÃO PARA IMPORTAR DATASET

dir_path = "data/"

def importaData(files):
    
    data = dir_path + files
    allFiles = glob.glob('data/*.csv')

    list_ = []
    cols = ['ANO_BO', 'DATAOCORRENCIA', 'PERIDOOCORRENCIA', 'BAIRRO', 'CIDADE', 'LATITUDE', 'LONGITUDE', 'LOGRADOURO', \
           'DESCRICAOLOCAL', 'DESCR_TIPO_VEICULO']

    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0, delimiter=';', usecols=cols)
        list_.append(df)

    df = pd.concat(list_, axis = 0, ignore_index = True)
    
    return df

# IMPORTANDO DATASET

all_data = importaData('*.csv')

/home/eduardo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
# VISUALIZANDO DATAFRAME

all_data.tail()

,ANO_BO,DATAOCORRENCIA,PERIDOOCORRENCIA,LOGRADOURO,BAIRRO,CIDADE,LATITUDE,LONGITUDE,DESCRICAOLOCAL,DESCR_TIPO_VEICULO
1304698,2015,31/12/2015,A NOITE,RUA CORONEL CAMISAO,BUTANTA,S.PAULO,"-23,57571329","-46,73134558",Via pública,AUTOMOVEL
1304699,2015,31/12/2015,A NOITE,RUA CAUBI,CAMPO LIMPO,S.PAULO,"-23,637218","-46,74504626",Via pública,NaN
1304700,2015,31/12/2015,A NOITE,RUA CAUBI,CAMPO LIMPO,S.PAULO,"-23,637218","-46,74504626",Via pública,AUTOMOVEL
1304701,2015,31/12/2015,A NOITE,RUA CAUBI,CAMPO LIMPO,S.PAULO,"-23,637218","-46,74504626",Via pública,NaN
1304702,2015,31/12/2015,A NOITE,RUA CAUBI,CAMPO LIMPO,S.PAULO,"-23,637218","-46,74504626",Via pública,AUTOMOVEL


In [5]:
# DESCRIÇÃO ESTATÍSTICA

all_data.describe()

,ANO_BO,DATAOCORRENCIA,PERIDOOCORRENCIA,LOGRADOURO,BAIRRO,CIDADE,LATITUDE,LONGITUDE,DESCRICAOLOCAL,DESCR_TIPO_VEICULO
count,1304703,1304702,1304702,1215343,1041685,1215345,1066431,1066429,1304701,302552
unique,14,4256,7,116080,34936,387,432700,433678,38,26
top,2014,06/05/2014,A NOITE,AVENIDA SAPOPEMBA,JABAQUARA,S.PAULO,"-23,50594973","-46,45693179",Via pública,AUTOMOVEL
freq,195621,1294,577242,4072,20177,1174304,988,988,895433,190430


In [6]:
print("Tipos de dados e valores 'missing': ")

all_data.info()

Tipos de dados e valores 'missing': 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304703 entries, 0 to 1304702
Data columns (total 10 columns):
ANO_BO                1304703 non-null object
DATAOCORRENCIA        1304702 non-null object
PERIDOOCORRENCIA      1304702 non-null object
LOGRADOURO            1215343 non-null object
BAIRRO                1041685 non-null object
CIDADE                1215345 non-null object
LATITUDE              1066431 non-null object
LONGITUDE             1066429 non-null object
DESCRICAOLOCAL        1304701 non-null object
DESCR_TIPO_VEICULO    302552 non-null object
dtypes: object(10)
memory usage: 99.5+ MB
